In [5]:
import sys
import os
import json
import logging
from datetime import datetime
from typing import Dict, List, Any, Optional, Union
import warnings
warnings.filterwarnings('ignore')

# Configure logging for clean, professional output
logging.basicConfig(
    level=logging.INFO,
    format='%(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)
logger = logging.getLogger(__name__)

%pip install transformers  --upgrade

logger.info("🧠 Welcome to LLM-Powered Agents!")
logger.info("=" * 60)

# Set up our workspace
current_dir = os.getcwd()
if 'notebooks' in current_dir:
    project_dir = os.path.dirname(current_dir)
else:
    project_dir = current_dir

logger.info("🔧 Environment Setup Complete!")
logger.info(f"📍 Project directory: {project_dir}")
logger.info(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

logger.info("\n🎯 Today's Journey:")
logger.info("   🔄 1. Compare Rule-Based vs LLM Approaches")
logger.info("   🧠 2. Build LLM Integration System") 
logger.info("   🛠️ 3. Create Intelligent Tool Selection")
logger.info("   💬 4. Test Natural Conversation")

logger.info("\n🚀 Let's give your agents real intelligence!")

# Note: We'll start with a simple LLM simulation, then show how to integrate real LLMs
logger.info("\n💡 LLM Options (in order of recommendation):")
logger.info("   🥇 Hugging Face (free, excellent models) - Unsloth DeepSeek-R1-Distill 4-bit")
logger.info("   🥈 Ollama (free, local, powerful) - ollama.ai")
logger.info("\n🔧 Hardware: Optimized for GPU/Colab - using Unsloth's 4-bit quantized model!")
logger.info("🚀 We'll auto-detect the best available option!")

Note: you may need to restart the kernel to use updated packages.
🧠 Welcome to LLM-Powered Agents!
🔧 Environment Setup Complete!
📍 Project directory: c:\Users\Sarthak\OneDrive\Documents\Desktop\Python projects\genai-workshop
📅 Session started: 2025-07-21 13:32:32

🎯 Today's Journey:
   🔄 1. Compare Rule-Based vs LLM Approaches
   🧠 2. Build LLM Integration System
   🛠️ 3. Create Intelligent Tool Selection
   💬 4. Test Natural Conversation

🚀 Let's give your agents real intelligence!

💡 LLM Options (in order of recommendation):
   🥇 Hugging Face (free, excellent models) - Unsloth DeepSeek-R1-Distill 4-bit
   🥈 Ollama (free, local, powerful) - ollama.ai

🔧 Hardware: Optimized for GPU/Colab - using Unsloth's 4-bit quantized model!
🚀 We'll auto-detect the best available option!


In [6]:
import random
import re
import json

# Initialize logging for model loading
logger.info("\n🚀 Initializing LLM Model...")
logger.info("=" * 50)

class SimpleLLMClient:
    """
    Simple LLM client that can work with multiple backends.
    Supports: OpenAI API, Ollama, or Hugging Face Transformers
    
    This shows how to integrate real LLMs into agent systems.
    """
    
    def __init__(self, backend="huggingface", model_name="auto"):
        """
        Initialize LLM client with specified backend.
        
        Args:
            backend: "huggingface", "openai", or "ollama"
            model_name: Model to use (depends on backend, "auto" for best available)
        """
        self.backend = backend
        self.model_name = model_name
        self.client = None
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        self.logger.info(f"🧠 Initializing LLM: {backend}")
        if model_name != "auto":
            self.logger.info(f"🎯 Requested model: {model_name}")
        
        # Initialize based on backend choice
        if backend == "huggingface":
            self._init_huggingface()
        elif backend == "ollama":
            self._init_ollama()
        else:
            raise Exception(f"Unsupported backend: {backend}. Use 'huggingface' or 'ollama'")
    
    def _init_huggingface(self):
        """Initialize Hugging Face transformers (optimized for GPU systems)"""
        try:
            from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
            import torch
            self.logger.info("🤗 Loading Hugging Face model...")
            
            # Check GPU availability (works on both local and Colab)
            if torch.cuda.is_available():
                device = "cuda"
                self.logger.info(f"🔧 Using device: GPU (CUDA)")
            else:
                device = "cpu"
                self.logger.info(f"🔧 Using device: CPU (will be slower)")
            
            # Use Unsloth's pre-quantized DeepSeek-R1-Distill model
            model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"
            
            self.logger.info(f"🔄 Loading model: {model_name}")
            self.logger.info(f"💡 Loading Unsloth's optimized 4-bit quantized version...")

            # 1) Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            
            # 2) Load pre-quantized model with Unsloth optimizations
            if device == "cuda":
                # GPU: Use the pre-quantized model as-is
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map="auto",
                    trust_remote_code=True
                )
            else:
                # CPU fallback: Load in float32
                self.logger.warning("⚠️ GPU not available, loading in CPU mode (slower)")
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float32,
                    device_map=None,
                    trust_remote_code=True
                ).to(device)
            
            self.model.eval()  # Set to evaluation mode
            
            self.logger.info(f"✅ Model loaded successfully!")
            self.model_name = model_name
            self.client = "unsloth_deepseek_loaded"  # Flag to indicate model is ready
                    
            if self.model is None:
                raise Exception("Model failed to load - check GPU memory")
            
        except ImportError:
            raise Exception("Required packages not installed. Run: pip install transformers torch bitsandbytes")
        except Exception as e:
            raise Exception(f"HuggingFace initialization failed: {e}")

    def _init_ollama(self):
        """Initialize Ollama local LLM (requires Ollama installed)"""
        try:
            import requests
            
            # Test if Ollama is running
            response = requests.get("http://localhost:11434/api/tags", timeout=5)
            if response.status_code == 200:
                self.client = "ollama_available"
                self.logger.info("✅ Ollama connection established!")
            else:
                raise Exception("Ollama not responding - make sure it's running")
                
        except Exception as e:
            raise Exception(f"Error connecting to Ollama: {e}. Make sure Ollama is installed and running: https://ollama.ai")
    
    def generate_response(self, prompt: str, max_tokens: int = 100) -> str:
        """
        Generate response using the configured LLM backend.
        
        Args:
            prompt: Input prompt for the LLM
            max_tokens: Maximum tokens to generate
            
        Returns:
            Generated response from the LLM
        """
        try:
            if self.backend == "huggingface":
                return self._generate_huggingface(prompt, max_tokens)
            elif self.backend == "ollama":
                return self._generate_ollama(prompt, max_tokens)
            else:
                raise Exception(f"Unknown backend: {self.backend}")
                
        except Exception as e:
            raise Exception(f"Error generating response: {e}")
    
    def _generate_huggingface(self, prompt: str, max_tokens: int) -> str:
        """Generate response using Unsloth's optimized DeepSeek-R1-Distill model"""
        if self.model is None or self.tokenizer is None:
            raise Exception("Unsloth DeepSeek model not properly loaded")
        
        try:
            import torch
            
            # DeepSeek uses standard chat format
            messages = [
                {"role": "system", "content": "You are a helpful AI learning assistant."},
                {"role": "user", "content": prompt}
            ]
            
            # Apply chat template and tokenize
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            # Tokenize the formatted text
            inputs = self.tokenizer(
                text, 
                return_tensors="pt", 
                truncation=True, 
                max_length=2048
            ).to(self.model.device)
            
            # Generate response with optimized parameters for quantized model
            with torch.inference_mode():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=min(max_tokens, 150),
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    repetition_penalty=1.1,
                    pad_token_id=self.tokenizer.eos_token_id,
                    use_cache=True  # Optimize for quantized model
                )
            
            # Decode only the new tokens (exclude input)
            input_length = inputs.input_ids.shape[1]
            new_tokens = outputs[0][input_length:]
            response = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
            
            # Clean up the response
            response = response.strip()
            
            # Remove any residual chat formatting
            response = response.replace("<|im_end|>", "").replace("<|im_start|>", "").strip()
            
            # Quality check
            if len(response) > 5 and not response.lower().startswith(prompt.lower()[:10]):
                return response
            else:
                return "I'd be happy to help you with that!"
            
        except Exception as e:
            raise Exception(f"Error in Unsloth DeepSeek generation: {e}")
    
    def _generate_ollama(self, prompt: str, max_tokens: int) -> str:
        """Generate response using Ollama"""
        import requests
        
        data = {
            "model": "llama3",  # Default model
            "prompt": prompt,
            "stream": False
        }
        
        response = requests.post("http://localhost:11434/api/generate", json=data)
        if response.status_code == 200:
            return response.json().get("response", "").strip()
        else:
            raise Exception(f"Ollama request failed with status {response.status_code}")

# 🎯 Initialize the LLM client once for the entire notebook
logger.info("🔄 Creating global LLM client...")
llm = SimpleLLMClient(backend="huggingface")

logger.info("✅ Model initialization complete!")
logger.info("🚀 Ready for agent development!")


🚀 Initializing LLM Model...
🔄 Creating global LLM client...
🧠 Initializing LLM: huggingface
🤗 Loading Hugging Face model...
🔧 Using device: CPU (will be slower)
🔄 Loading model: unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit
💡 Loading Unsloth's optimized 4-bit quantized version...
⚠️ GPU not available, loading in CPU mode (slower)


Exception: Required packages not installed. Run: pip install transformers torch bitsandbytes

In [ ]:

import random
import re
import json

class SimpleLLMClient:
    """
    Simple LLM client that can work with multiple backends.
    Supports: OpenAI API, Ollama, or Hugging Face Transformers
    
    This shows how to integrate real LLMs into agent systems.
    """
    
    _instance = None  # Class variable to store singleton instance
    
    def __new__(cls, backend="huggingface", model_name="auto"):
        # Implement singleton pattern to prevent duplicate initialization
        if cls._instance is None:
            cls._instance = super(SimpleLLMClient, cls).__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self, backend="huggingface", model_name="auto"):
        """
        Initialize LLM client with specified backend.
        
        Args:
            backend: "huggingface", "openai", or "ollama"
            model_name: Model to use (depends on backend, "auto" for best available)
        """
        # Prevent re-initialization
        if hasattr(self, '_initialized') and self._initialized:
            return
            
        self.backend = backend
        self.model_name = model_name
        self.client = None
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        self.logger.info(f"🧠 Initializing LLM: {backend}")
        if model_name != "auto":
            self.logger.info(f"🎯 Requested model: {model_name}")
        
        # Initialize based on backend choice
        if backend == "huggingface":
            self._init_huggingface()
        elif backend == "ollama":
            self._init_ollama()
        else:
            raise Exception(f"Unsupported backend: {backend}. Use 'huggingface' or 'ollama'")
            
        self._initialized = True
    
    def _init_huggingface(self):
        """Initialize Hugging Face transformers (optimized for GPU systems)"""
        try:
            from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
            import torch
            self.logger.info("🤗 Loading Hugging Face model...")
            
            # Check GPU availability (works on both local and Colab)
            if torch.cuda.is_available():
                device = "cuda"
                self.logger.info(f"🔧 Using device: GPU (CUDA)")
            else:
                device = "cpu"
                self.logger.info(f"🔧 Using device: CPU (will be slower)")
            
            # Use Unsloth's pre-quantized DeepSeek-R1-Distill model
            model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-bnb-4bit"
            
            self.logger.info(f"🔄 Loading model: {model_name}")
            self.logger.info(f"💡 Loading Unsloth's optimized 4-bit quantized version...")

            # 1) Load tokenizer
            self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
            
            # 2) Load pre-quantized model with Unsloth optimizations
            if device == "cuda":
                # GPU: Use the pre-quantized model as-is
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    device_map="auto",
                    trust_remote_code=True
                )
            else:
                # CPU fallback: Load in float32
                self.logger.warning("⚠️ GPU not available, loading in CPU mode (slower)")
                self.model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    torch_dtype=torch.float32,
                    device_map=None,
                    trust_remote_code=True
                ).to(device)
            
            self.model.eval()  # Set to evaluation mode
            
            self.logger.info(f"✅ Model loaded successfully!")
            self.model_name = model_name
            self.client = "unsloth_deepseek_loaded"  # Flag to indicate model is ready
                    
            if self.model is None:
                raise Exception("Model failed to load - check GPU memory")
            
        except ImportError:
            raise Exception("Required packages not installed. Run: pip install transformers torch bitsandbytes")
        except Exception as e:
            raise Exception(f"HuggingFace initialization failed: {e}")

            
    def _init_ollama(self):
        """Initialize Ollama local LLM (requires Ollama installed)"""
        try:
            import requests
            
            # Test if Ollama is running
            response = requests.get("http://localhost:11434/api/tags", timeout=5)
            if response.status_code == 200:
                self.client = "ollama_available"
                self.logger.info("✅ Ollama connection established!")
            else:
                raise Exception("Ollama not responding - make sure it's running")
                
        except Exception as e:
            raise Exception(f"Error connecting to Ollama: {e}. Make sure Ollama is installed and running: https://ollama.ai")
    
    def generate_response(self, prompt: str, max_tokens: int = 100) -> str:
        """
        Generate response using the configured LLM backend.
        
        Args:
            prompt: Input prompt for the LLM
            max_tokens: Maximum tokens to generate
            
        Returns:
            Generated response from the LLM
        """
        # Don't print during generation to avoid clutter
        # print(f"🧠 LLM generating response...")
        
        try:
            if self.backend == "huggingface":
                return self._generate_huggingface(prompt, max_tokens)
            elif self.backend == "ollama":
                return self._generate_ollama(prompt, max_tokens)
            else:
                raise Exception(f"Unknown backend: {self.backend}")
                
        except Exception as e:
            raise Exception(f"Error generating response: {e}")
    
    def _generate_huggingface(self, prompt: str, max_tokens: int) -> str:
        """Generate response using Unsloth's optimized DeepSeek-R1-Distill model"""
        if self.model is None or self.tokenizer is None:
            raise Exception("Unsloth DeepSeek model not properly loaded")
        
        try:
            import torch
            
            # DeepSeek uses standard chat format
            messages = [
                {"role": "system", "content": "You are a helpful AI learning assistant."},
                {"role": "user", "content": prompt}
            ]
            
            # Apply chat template and tokenize
            text = self.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )
            
            # Tokenize the formatted text
            inputs = self.tokenizer(
                text, 
                return_tensors="pt", 
                truncation=True, 
                max_length=2048
            ).to(self.model.device)
            
            # Generate response with optimized parameters for quantized model
            with torch.inference_mode():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=min(max_tokens, 150),
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    repetition_penalty=1.1,
                    pad_token_id=self.tokenizer.eos_token_id,
                    use_cache=True  # Optimize for quantized model
                )
            
            # Decode only the new tokens (exclude input)
            input_length = inputs.input_ids.shape[1]
            new_tokens = outputs[0][input_length:]
            response = self.tokenizer.decode(new_tokens, skip_special_tokens=True)
            
            # Clean up the response
            response = response.strip()
            
            # Remove any residual chat formatting
            response = response.replace("<|im_end|>", "").replace("<|im_start|>", "").strip()
            
            # Quality check
            if len(response) > 5 and not response.lower().startswith(prompt.lower()[:10]):
                return response
            else:
                return "I'd be happy to help you with that!"
            
        except Exception as e:
            raise Exception(f"Error in Unsloth DeepSeek generation: {e}")
    
    def _generate_ollama(self, prompt: str, max_tokens: int) -> str:
        """Generate response using Ollama"""
        import requests
        
        data = {
            "model": "llama3",  # Default model
            "prompt": prompt,
            "stream": False
        }
        
        response = requests.post("http://localhost:11434/api/generate", json=data)
        if response.status_code == 200:
            return response.json().get("response", "").strip()
        else:
            raise Exception(f"Ollama request failed with status {response.status_code}")

# 🧪 Test the real LLM integration
logger.info("\n🧪 Testing Real LLM Integration")
logger.info("-" * 40)

# Create LLM client (will auto-detect best available option)
logger.info("\n🔄 Initializing LLM client...")

llm = SimpleLLMClient(backend="huggingface")

logger.info("\n🧪 Running tests...")

# Test with educational prompts
test_prompts = [
    "Hello! I'm new to programming and want to learn Python.",
    "Can you explain what a function is in simple terms?"
]

for i, prompt in enumerate(test_prompts, 1):
    logger.info(f"\n📝 Test {i}: {prompt}")
    
    # Generate response using real LLM
    response = llm.generate_response(prompt, max_tokens=150)
    
    logger.info(f"🤖 Response: {response}")

logger.info(f"\n✅ LLM integration complete! The agent now has genuine AI intelligence.")
logger.info(f"🚀 Next: We'll integrate this into our agent architecture!")

In [ ]:

class LLMToolSelector:
    """
    Uses real LLM intelligence to choose the best tools for each task.
    This is much smarter than rule-based selection!
    """
    
    def __init__(self, llm_client: SimpleLLMClient):
        self.llm = llm_client
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        # Available tools with descriptions (this is what the LLM sees)
        self.available_tools = {
            'calculator': 'Performs mathematical calculations and computations',
            'file_manager': 'Creates, reads, and manages files and documents', 
            'timer': 'Sets timers and manages time-based activities',
            'web_search': 'Searches the internet for current information',
            'calendar': 'Manages schedules, appointments, and deadlines'
        }
    
    def select_tools(self, user_request: str, context: str = "") -> List[str]:
        """
        Use LLM to intelligently select tools based on user request.
        
        Args:
            user_request: What the user wants to accomplish
            context: Additional context about the user or situation
            
        Returns:
            List of tools the LLM recommends using
        """
        self.logger.debug(f"🛠️ Selecting tools for: '{user_request}'")
        
        # Create a structured prompt for tool selection
        tool_list = "\n".join([f"- {name}: {desc}" for name, desc in self.available_tools.items()])
        
        prompt = f"""You are an AI agent that helps users by selecting the right tools.

User Request: "{user_request}"
Context: {context}

Available Tools:
{tool_list}

Task: Analyze the user's request and select which tools would be most helpful. Consider:
1. What is the user trying to accomplish?
2. What information or actions are needed?
3. Which tools can provide that?

Respond with only the tool names, separated by commas. For example: "calculator, file_manager"
Only choose the tools that are absolutely necessary for this request. 
Selected tools:"""

        # Get LLM response
        response = self.llm.generate_response(prompt, max_tokens=500)
        
        # Parse the LLM's tool selection
        selected_tools = []
        for tool_name in self.available_tools.keys():
            if tool_name in response.lower():
                selected_tools.append(tool_name)
        
        self.logger.info(f"🎯 Selected tools: {selected_tools}")
        return selected_tools
    
    def explain_selection(self, user_request: str, selected_tools: List[str]) -> str:
        """
        Ask the LLM to explain why it selected these tools.
        This helps users understand the agent's reasoning!
        """
        if not selected_tools:
            return "No tools were needed for this request."
        
        tools_str = ", ".join(selected_tools)
        
        prompt = f"""Explain in one sentence why these tools ({tools_str}) are the best choice for this request: "{user_request}"

Explanation:"""
        
        explanation = self.llm.generate_response(prompt, max_tokens=60)
        return explanation.strip()

# 🧪 Test LLM-powered tool selection
logger.info("\n🧪 Testing LLM Tool Selection")
logger.info("-" * 40)

# Create tool selector with our LLM
tool_selector = LLMToolSelector(llm)

# Test various requests to see intelligent tool selection
test_requests = [
    "I need to calculate the compound interest on my savings",
    "Help me create a study schedule for learning Python",
    "I want to save my class notes somewhere organized", 
    "What's the latest information about machine learning trends?",
    "Set up a 25-minute focus session for studying"
]

for request in test_requests:
    logger.info(f"\n📝 User Request: {request}")
    
    # Let LLM select tools
    selected_tools = tool_selector.select_tools(request)
    
    # Get explanation
    explanation = tool_selector.explain_selection(request, selected_tools)
    
    logger.info(f"🤖 Reasoning: {explanation}")

logger.info(f"\n✅ Notice how the LLM:")
logger.info(f"   • Understands the intent behind each request")
logger.info(f"   • Selects appropriate tools intelligently")
logger.info(f"   • Can explain its reasoning")
logger.info(f"   • Handles requests it's never seen before!")

logger.info(f"\n🚀 This is the power of LLM-driven agent intelligence!")

In [ ]:

class LLMAgent:
    """
    A complete AI agent powered by real LLM intelligence.
    This combines all our previous concepts with genuine AI reasoning.
    """
    
    def __init__(self, name: str = "LLMAgent", llm_client: SimpleLLMClient = None):
        self.name = name
        self.logger = logging.getLogger(f"{__name__}.{self.__class__.__name__}")
        
        # Use provided LLM client or create new one
        if llm_client is not None:
            self.llm = llm_client
            self.logger.info(f"🤖 {self.name} initialized with shared LLM client!")
        else:
            # Initialize LLM and tool systems
            self.llm = SimpleLLMClient(backend="huggingface")
            self.logger.info(f"🤖 {self.name} initialized with new LLM client!")
            
        self.tool_selector = LLMToolSelector(self.llm)
        
        # Simple memory for conversation context
        self.conversation_history = []
    
    def process_request(self, user_input: str) -> str:
        """
        Process user request using LLM-powered intelligence.
        This is the complete agent interaction loop!
        
        Args:
            user_input: What the user said
            
        Returns:
            Intelligent response from the agent
        """
        self.logger.info(f"🤖 {self.name} processing: '{user_input}'")
        
        # STEP 1: 👁️ PERCEPTION - Add to conversation context
        self.conversation_history.append(f"User: {user_input}")
        context = " ".join(self.conversation_history[-3:])  # Last 3 interactions
        
        # STEP 2: 🧠 LLM REASONING - Tool selection
        self.logger.debug("🛠️ Selecting tools...")
        selected_tools = self.tool_selector.select_tools(user_input, context)
        
        # STEP 3: 🤚 ACTION - Generate response
        self.logger.debug("💬 Generating response...")
        response = self._generate_contextual_response(user_input, selected_tools, context)
        
        # STEP 4: 💾 MEMORY - Store interaction
        self.conversation_history.append(f"Agent: {response}")
        
        self.logger.info(f"🤖 {self.name}: {response}")
        return response
    
    def _generate_contextual_response(self, user_input: str, tools: List[str], context: str) -> str:
        """
        Generate a natural response using LLM with tool awareness.
        """
        # Create a comprehensive prompt for natural conversation
        tools_info = f"Available tools: {', '.join(tools)}" if tools else "No tools needed"
        
        prompt = f"""You are a helpful AI learning assistant having a conversation.

Recent context: {context}
Current user message: "{user_input}"
{tools_info}

Respond naturally and helpfully. If tools were selected, mention how you would use them to help. Keep response conversational and encouraging.

Response:"""
        
        response = self.llm.generate_response(prompt, max_tokens=100)
        
        # Add tool usage information if tools were selected
        if tools:
            tool_explanation = self.tool_selector.explain_selection(user_input, tools)
            response += f" I'll use {', '.join(tools)} to help - {tool_explanation.lower()}"
        
        return response.strip()

# 🧪 Test the complete LLM-powered agent
logger.info("\n🧪 Testing Complete LLM Agent")
logger.info("-" * 40)

# Create our intelligent agent using the existing LLM client (no duplication!)
smart_agent = LLMAgent("StudyBuddy", llm_client=llm)

# Have a real conversation with the LLM-powered agent
conversation = [
    "Hello! I'm struggling with learning Python programming",
    "I need help with loops specifically - they're confusing me",
    "Can you create a study plan for me?",
    "What's 15% of 240? I need this for my homework", 
    "Thanks! You've been really helpful"
]

for message in conversation:
    logger.info(f"\n👤 User: {message}")
    response = smart_agent.process_request(message)

logger.info(f"\n🎉 Complete LLM-powered agent success!")
logger.info(f"✅ Real LLM intelligence")
logger.info(f"✅ Intelligent tool selection") 
logger.info(f"✅ Natural conversation")
logger.info(f"✅ Context awareness")
logger.info(f"✅ Memory of past interactions")

logger.info(f"\n💡 This agent can handle ANY request intelligently!")
logger.info(f"🚀 Ready for the StudyBuddy multi-agent system on Day 4!")